In [13]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [14]:
data = pd.read_excel('lecture_portfolio.xlsx')
data.set_index('Date', inplace=True)
data

AAL      CSCO         F        GE       JPM       NKE  \
Date                                                                     
2009-05-31 -0.318790 -0.041972 -0.037990  0.066084  0.118654  0.087760   
2009-06-30 -0.058739  0.007509  0.055053 -0.124007 -0.076210 -0.089057   
2009-07-31  0.205789  0.180188  0.317985  0.143372  0.134790  0.093886   
2009-08-31  0.159974 -0.019063 -0.050436  0.036878  0.124015 -0.022504   
2009-09-30  0.382362  0.089824 -0.051307  0.188294  0.008293  0.173593   
...              ...       ...       ...       ...       ...       ...   
2021-08-31 -0.021409  0.066101 -0.065769  0.017652  0.054009 -0.014732   
2021-09-30  0.029087 -0.077770  0.086723 -0.021818  0.023382 -0.118429   
2021-10-31 -0.064056  0.035511  0.206486  0.018130  0.044402  0.152169   
2021-11-30 -0.078193 -0.019737  0.129646 -0.093759 -0.064648  0.012109   
2021-12-31  0.020079  0.166030  0.071120 -0.001503 -0.001972 -0.001197   

                 SPY       UAL       XOM  
Date                                      
2009-05-31  0.058925 -0.054407  0.046877  
2009-06-30 -0.001254 -0.314578  0.007476  
2009-07-31  0.074633  0.291563  0.006893  
2009-08-31  0.036504  0.511701 -0.012051  
2009-09-30  0.035466  0.479945 -0.007800  
...              ...       ...       ...  
2021-08-31  0.029941 -0.004314 -0.038464  
2021-09-30 -0.046605  0.022791  0.078870  
2021-10-31  0.070435 -0.029789  0.096327  
2021-11-30 -0.007582 -0.083639 -0.058890  
2021-12-31  0.051534  0.051077  0.021620  

[152 rows x 9 columns]

# OLS Regression


In [15]:
# OLS Linear Regression
y_data = data['NKE']
x_data = data['SPY']

# Adds the X (Independent) variable to the regression
X = sm.add_constant(x_data)

# Run Regression
results = sm.OLS(y_data, X, missing='drop').fit()

In [16]:
# Regression Statistics

# Summary
summary = results.summary()

# All the Alpha and Beta Statistics
params = results.params
alpha = params[0]
beta = params[1]

# R-Squared
r_squared = results.rsquared

# Info Ratio Annualized
info_ratio = alpha / results.resid.std() * np.sqrt(12)

# Treynor Ratio Annualized
treynor_ratio = y_data.mean() / beta * 12


print(f"Alpha: {alpha}")
print(f"Beta: {beta}")
print(f"R-Squared: {r_squared}")
print(f"Info Ratio: {info_ratio}")
print(f"Treynor Ratio: {treynor_ratio}")

Alpha: 0.008413607037696617
Beta: 0.8480297934356753
R-Squared: 0.272883020162199
Info Ratio: 0.5278101863540234
Treynor Ratio: 0.2781393875229979


# Pricing

Using spy as the factor predicts the return of Nike with decent accuracy. Only understating .8% of a return using SPY as teh factor Should be 0 if it priced perfectly.

# Hedging

In [29]:
# OLS Linear Regression
y_data = data['NKE']
x_data = data['SPY']

# Adds the X (Independent) variable to the regression
X = sm.add_constant(x_data)

# Run Regression
results = sm.OLS(y_data, X, missing='drop').fit()
print(results.params[1])

0.8480297934356753


Go short .848 dollars of SPY for every 1 dollar we want to hedge out on GE

In [17]:
# OLS Linear Regression Iterating over multiple data values

stocks = data.columns

data_list = []

X = sm.add_constant(data['SPY'])
for stock in stocks:
    # Regression for specific stock (stock) with SPY as independent
    
    y = data[stock]
    results = sm.OLS(y, X, missing='drop').fit()
    
    # Alpha, Beta, and R-Squared
    params = results.params
    alpha = params[0] * 12
    beta = params[1]
    r_squared = results.rsquared
    
    # Info ratio
    info_ratio = (alpha / 12) / results.resid.std() *np.sqrt(12)
    
    # Treynor Ratio
    treynor_ratio = y.mean() / beta * 12
    
    # Adding data to DataFrame
    frame = {"Stock":stock, "Alpha":alpha, "Beta":beta, 
             "R-Squared":r_squared, "Info Ratio":info_ratio, 
            "Treynor Ratio":treynor_ratio}
    data_list.append(frame)
    
df = pd.DataFrame(data_list)
df.set_index('Stock', inplace=True)
df

Alpha      Beta  R-Squared  Info Ratio  Treynor Ratio
Stock                                                              
AAL    4.827873e-02  1.281696   0.116422    0.098948       0.196751
CSCO  -3.695119e-02  1.152453   0.395743   -0.187776       0.127020
F     -3.485451e-02  1.319757   0.317470   -0.130345       0.132673
GE    -1.297606e-01  1.228395   0.299097   -0.499369       0.053449
JPM   -3.563630e-02  1.328296   0.533860   -0.207775       0.132255
NKE    1.009633e-01  0.848030   0.272883    0.527810       0.278139
SPY   -2.081668e-17  1.000000   1.000000   -0.714853       0.159083
UAL    7.382712e-02  1.345892   0.139548    0.159861       0.213937
XOM   -1.188681e-01  1.066450   0.438083   -0.712207       0.047622

# Multivariate Regression and Replication

In [20]:
# Multivariate regression

# Adds AAL, CSCO, F, GE, and JPM as the factors in the multivariate regression
X = sm.add_constant(data[list(data.columns)[:5]])
y = data['SPY']

# Regresses Spy onto all the assets
results = sm.OLS(y, X, missing='drop').fit()


# Alpha, Beta, and R-Squared
params = results.params
alpha = params[0] * 12
beta = params[1:]
r_squared = results.rsquared

print(f"Alpha: {alpha}")
print(f"R-Squared: {r_squared}")
print(f"Info Ratio: {info_ratio}")
print(f"Treynor Ratio: {treynor_ratio}")
print("Betas")
print(beta)

Alpha: 0.07701758661949451
R-Squared: 0.6658086754175105
Info Ratio: -0.7122067182174061
Treynor Ratio: 0.047621569030617164
Betas
AAL     0.011164
CSCO    0.164591
F       0.059599
GE      0.061501
JPM     0.231585
dtype: float64


# Replication

Using the example above. We ran a multivariate regression of SPY onto the assets American Airlines, Cisco, Ford, General Eletric, and JP morgan.
To replicate the returns of SPY we would invest Beta dollars of each asset for every 1 dollar of SPY we would like to replicate.